In [6]:
import numpy as np
import cv2
import pandas as pd

In [7]:
video_name = 'Help'
data = pd.read_fwf('data/head_tracking_data/Subject_6/' + video_name + '.txt', header=None)

data.columns = ['latitude', 'longitude']

cap = cv2.VideoCapture('data/video_files/' + video_name + '.mp4')

# Get video meta data
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculate pixels per degree
width_normalized = width/360
height_normalized = height/180

# Calculate FoV offsets, 110 degrees horizontal, 100 degrees verti
width_offset = width_normalized*55
height_offset = height_normalized*50

# Here we figure out the spacing of the video and dataset,
# Sometimes there are more frames than data points, so using calc_mod
# We can make sure the data points are spaced evenly across the frames
calc_mod = round(1/(round((length/len(data)),2)-1))

In [8]:
tiling_x = 4
tiling_y = 4

In [9]:
def conversion(lat, lon):
    x = (180 - lon) * width_normalized
    y = (90 - lat) * height_normalized
    return(x, y)

def start_end_point(x, y):
    start_point = (int(x - width_offset), int(y + height_offset))
    end_point = (int(x + width_offset), int(y - height_offset))
    return(start_point, end_point)

index = 0
skip = True
zones = [i for i in range(1,61)]

# so we only need to re-run this part of code to show video again
cap = cv2.VideoCapture('data/video_files/' + video_name + '.mp4')
ret, frame = cap.read()
cv2.namedWindow('frame', 0)
cv2.resizeWindow('frame', 1920,1080)

cv2.imshow('frame',cv2.flip(frame,1))
go = False

while(cap.isOpened()):
    
    while(not go): # just to make it so we need to press a button to start the video
        cv2.waitKey(0)

        if(ord('a')):
            go = True

    ret, frame = cap.read()

    if (((index % calc_mod) == 0) and skip):
        skip = not skip
        index = index - 1
    
    # Calculate focus point
    focus_x, focus_y = conversion(data.iloc[index].latitude, data.iloc[index].longitude)
    # Calculate top left and bottom right of FoV based on focus point
    start_point, end_point = start_end_point(focus_x, focus_y)
    # Draw FoV rectangle
    cv2.rectangle(frame, start_point, end_point, (0, 255, 0), 10)

    # Draw tiles grid
    for i in zones:
        cv2.line(frame, (int((frame.shape[1]/tiling_x) * i), 0), (int((frame.shape[1]/tiling_x) * i), frame.shape[0]), (255,255,255), 2)
        cv2.line(frame, (0, int(frame.shape[0]/tiling_y) * i), (frame.shape[1], int(frame.shape[0]/tiling_y) * i), (255,255,255), 2)

    cv2.imshow('frame',cv2.flip(frame,1))

    index += 1
    if((index % calc_mod) != 0):
        skip = True

    if (cv2.waitKey(1) & 0xFF == ord('q') or index > len(data)-1):
        break

cap.release()
cv2.destroyAllWindows()